In [10]:
import numpy as np
import pandas as pd
import redis

In [11]:
# Connect to redis client

hostname = 'localhost'
port = 6379

r = redis.StrictRedis(host=hostname, port=port)

In [7]:
r.ping()

True

In [8]:
r.keys()

[b'bull:AUTOMATION_QUEUE:completed',
 b'bull:AUTOMATION_QUEUE:2',
 b'bull:AUTOMATION_QUEUE:1',
 b'bull:AUTOMATION_QUEUE:id',
 b'attendance:logs',
 b'academy:register']

In [13]:
# Load numpy
np_file = np.load('data_frame_students_teachers.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values, columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.43222713, 1.074822, -0.12857957, -1.8332443..."
1,Barack Obama,Teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ..."
2,Chris Evans,Student,"[0.72012275, -1.4683539, -0.77757514, -1.46026..."
3,Morgan Freeman,Teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711..."
4,Scarlett Johansson,Student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578..."


In [15]:
df['name_role'] = df['Name'] + '@' + df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.43222713, 1.074822, -0.12857957, -1.8332443...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.72012275, -1.4683539, -0.77757514, -1.46026...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578...",Scarlett Johansson@Student


In [12]:
records = df[['name_role', 'Facial_Features']].to_dict(orient='records')
len(records)

5

### Save Records to Redis
- redis hashes
- keyname = 'academy:register'

In [16]:
vector = None
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis
    r.hset(name='academy:register', key=name_role, value=vector_bytes)

In [19]:
vector.dtype

dtype('float32')

### Retrieve Records from Redis

In [4]:
retrieve_dict = r.hgetall(name='academy:register')
retrieve_series = pd.Series(retrieve_dict)
retrieve_series

b'Chris Evans@Student'           b'\xf7Y8?\x05\xf3\xbb\xbf*\x0fG\xbf\x13\xea\xb...
b'Barack Obama@Teacher'          b'\xa5\xd7r?\xf6\xef\x0e\xbf\x96?\xc2=\x0f\xcb...
b'Angelina Jolie@Student'        b'\xe0L\xdd>\xc4\x93\x89?]\xaa\x03\xbe\xc0\xa7...
b'Morgan Freeman@Teacher'        b'\xfa\xb9\xf8>\x8e\x8b\x1e\xbf\xeb\x88\xd0\xb...
b'Scarlett Johansson@Student'    b'\xc6\xe5C>\xfd8\xfa\xbe\xb3_\xd8\xbf\xd6\x89...
dtype: object

In [5]:
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode(), index))

In [6]:
retrieve_series.index = index

In [7]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['name_role', 'facial_features']
retrieve_df

,name_role,facial_features
0,Chris Evans@Student,"[0.72012275, -1.4683539, -0.77757514, -1.46026..."
1,Barack Obama@Teacher,"[0.948603, -0.558349, 0.09484784, -1.2796344, ..."
2,Angelina Jolie@Student,"[0.43222713, 1.074822, -0.12857957, -1.8332443..."
3,Morgan Freeman@Teacher,"[0.4857939, -0.61931694, -0.4072946, 1.0686711..."
4,Scarlett Johansson@Student,"[0.1913062, -0.48871604, -1.6904205, 0.4424578..."


#### Registration Form

In [8]:
import cv2
from insightface.app import FaceAnalysis

In [9]:
# Configure
faceapp = FaceAnalysis(name='buffalo_sc', root='insightface_model', providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


#### Step1: Collect Person And Role

In [10]:
# Input
person_name = input('Enter your name: ')
trials = 3
role = 'Student'
for i in range(trials):
    role = input("""
        Please choose
        1. Student
        2. Teacher
        
        Enter number either 1 or 2
    """)
    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry, try again!')
        if i == 3:
            print('Exceed maximum trials')

key = person_name +'@'+ role


In [11]:
person_name, role

('Momo', 'Teacher')

#### Step2: Collect embedding faces

In [14]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embedding = []
sample = 0

while True:
    success, frame = cap.read()
    if not success:
        print('Unable to read camera')
        break
    # get results from insightface model
    results = faceapp.get(frame, max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
        
        # facial features
        embedding = res['embedding']
        face_embedding.append(embedding)
    
    if sample >= 200:
        break
        
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [15]:
len(face_embedding)

216

#### Step3: Optimize Data & Save in Redis

In [17]:
x_mean = np.asarray(face_embedding).mean(axis=0)
x_mean.shape

(512,)

In [18]:
# covert into bytes
x_mean_bytes = x_mean.tobytes()
x_mean_bytes

b'%L\xa2\xbe3\x17\xa7\xbfH\\4\xbfb\x05\xba\xbe\x17\xdal\xbfK\xa4??\xe6\xcc\xf6<\xc9w\xb2\xbf!\xdf[\xbe\x17\x99\xc5>\x95Q\x06?&G\x08=~\x9dw?\xb5\xb3&?\xef\x8dU\xbem0\xeb\xbd\x17OV=\xe1\xd4\x0b?\xb1\x15\x82\xbf\x01g\xd5\xbe\xb9q\x9b?5@5\xbf{M5\xbeW\xf8\xb8\xbf\x08\t\xb9\xbf\xcbd\xf4>\xf4S\xaf?\xcb\xcf\x8c\xbfO\xae\x86\xbe\x1a\xd2\xfb>\xe4\x13>?\xdaOS\xbe8\xee\x8a\xbc\xd1x\\?R\x19\xc2=N&\xc5\xbc\x9f)\x0c@\x13\xb29?\xac{\xb8\xbe\xbcM\xe8\xbe\x9dT\x96\xbfq\\\x0f>\xb91I\xbc\xf5i\xa0?\x8dy\xd3>\x00\x06\x8f:?9H\xbek\xfd\x9f\xbf\x80\xdf\x8c=\x9e\xca\xad>\x14?\xc0\xbf\xd7\xf3y>\xf0*\t\xc0\t\xb5\xe2\xbe\x1e\xaaA\xbf.\xec\x11?\n\xb4\x07\xc0\x9f\x90\xb4=\xd6<\x12\xbfzge\xbeT6\x92=\xc0\x17\xe8>\x98\x05v?g(%?eq\xb9?W \xc8\xbf\xa1%\x87?\x9c\n\x9c\xbf\xce\xcc\x90\xbf\xe9\xd9\x90>\x86i=?uS(\xbf\\\xa5\x98?\xec\x97\xb3\xbf\x9cX\x83\xbe\x9b\x84\xc2\xbfE6\t?% \x8b?\xe4s\x1e\xbf\x0c\xa9\xfb\xbd\xd9%S\xbf\xd3\xf4\xdd\xbe\x08\x8a??u\\\xdf\xbe\x0e8;\xbf\xe0\x81\xc1\xbfp\xf2\x91\xbfj\xf4\x81\xbf|\xd6X\xbf\x11a\x

In [19]:
# save key and facial bytes
r.hset(name='academy:register',key=key,value=x_mean_bytes)


1